In [1]:
import pyspark
pyspark.__version__

'3.1.3'

In [2]:
import findspark
findspark.init()
findspark.find()

'C:\\spark\\spark-3.1.3-bin-hadoop2.7'

In [3]:
# Import SparkSession
from pyspark.sql import SparkSession

# Create SparkSession 
spark = SparkSession.builder.appName("Adult_Data").getOrCreate() 
spark

In [4]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [5]:
class PrepareData:
    #Préparation des données
    def dataFrames(self, dataBrute):
        df = spark.read.csv(dataBrute, inferSchema=True, header=True)
        return df

    
#Pour combiner toutes les données d'entités et séparer les données "d'étiquettes" 
#dans un jeu de données, nous utilisons VectorAssembler.
    def assembler(self,df, inputColss, outputs):
        assembler =  VectorAssembler(inputCols=inputColss, outputCol=outputs)
        outputss = assembler.transform(df)
        return outputss
    
    
#Ensuite, nous diviserons les données en parties d'entraînement et de test.
    def model(self,model_df, train, test):
        training_df, test_df = model_df.randomSplit([train, test])
        return training_df, test_df
    
    
#Vérification de la prédiction et de la précision
#Ensuite, nous allons définir le modèle de classificateur d'arbre de décision en utilisant la classe 
#DecisionTreeClassifier et ajuster le modèle sur les données de train. Nous pouvons prédire les 
#données de test en utilisant la méthode trasnform().
    def df_classifier(self, training_dfs,test_dfs):
        df_classifier = DecisionTreeClassifier(labelCol="Classification").fit(training_dfs)
        df_predictions = df_classifier.transform(test_dfs)
        return df_classifier, df_predictions

#Après avoir formé le modèle, nous allons prédire les données de test et vérifier les mesures de 
#précision. Ici, nous pouvons utiliser MulticlassClassificationEvaluator pour vérifier l'exactitude. 
#La matrice de confusion peut être créée en utilisant la fonction confusion_matrix du module    
    def df_accuracy(self, labelCols ,metricNames, df_prediction):
        df_accuracy = MulticlassClassificationEvaluator(labelCol=labelCols, metricName=metricNames).evaluate(df_prediction)
        return df_accuracy


In [7]:
class Test:       
    classMain= PrepareData()
    df = classMain.dataFrames("dataR2.csv")
    df.show()
    
    print("``````````````````````````````````````````````````````````````````````````````````````````````````````````````````")
    
    df.groupBy('Classification').count().show()
    
    print("//////////////////////////////////////////////////////////////////////////////////////////////////////////////////")
    
    print(df.count())
    
    print("////////////////////////////////////////////////////////////////////////////////////////////////////////////////")
    
    print(df.head(3))

    print("//////////////////////////////////////////////////////////////////////////////////////////////////////////////////") 
    
    print(len(df.columns))
    
    print("//////////////////////////////////////////////////////////////////////////////////////////////////////////////////")
    
    print("Le nombre de colonne de dataframe est de :   ", df.columns)
    
    print("//////////////////////////////////////////////////////////////////////////////////////////////////////////////////")
    
    df.printSchema()
    
    print("/////////////////////////////////////////////////////////////////////////////////////////////////////////////////") 
    
    df.describe().show()
    
    print("/////////////////////////////////////////////////////////////////////////////////////////////////////////////////")
    
    
    
    assembler= classMain.assembler(df, ['Age','BMI','Glucose','Insulin','HOMA','Leptin','Adiponectin','Resistin','MCP'],"features")
    assembler.show()
    assemblers = assembler.select("features","Classification")
    
    print("//////////////////////////////////////////////////////////////////////////////////////////////////////////////////")
    
    l= PrepareData()
    training_df, test_df = l.model(assemblers,0.7,0.3)

    print(training_df.count())
    print(test_df.count())        
    
    print("//////////////////////////////////////////////////////////////////////////////////////////////////////////////////")
    
    df_classifiers, df_predictions = l.df_classifier(training_df,test_df)
    
    print(df_classifiers.featureImportances)
    
    print("//////////////////////////////////////////////////////////////////////////////////////////////////////////////////")
    df_predictions.show()
    print("//////////////////////////////////////////////////////////////////////////////////////////////////////////////////")
    
    l= PrepareData()
    df_acc = l.df_accuracy("Classification","accuracy",df_predictions)
    print(df_acc)
    print("//////////////////////////////////////////////////////////////////////////////////////////////////////////////////")     
        
        

+---+-----------+-------+-------+-----------+-------+-----------+--------+--------+--------------+
|Age|        BMI|Glucose|Insulin|       HOMA| Leptin|Adiponectin|Resistin|     MCP|Classification|
+---+-----------+-------+-------+-----------+-------+-----------+--------+--------+--------------+
| 48|       23.5|     70|  2.707|0.467408667| 8.8071|     9.7024| 7.99585| 417.114|             1|
| 83|20.69049454|     92|  3.115|0.706897333| 8.8438|   5.429285| 4.06405| 468.786|             1|
| 82|23.12467037|     91|  4.498|1.009651067|17.9393|   22.43204| 9.27715| 554.697|             1|
| 68|21.36752137|     77|  3.226|0.612724933| 9.8827|    7.16956|  12.766|  928.22|             1|
| 86|21.11111111|     92|  3.549|  0.8053864| 6.6994|    4.81924|10.57635|  773.92|             1|
| 49|22.85445769|     92|  3.226|0.732086933| 6.8317|   13.67975| 10.3176|  530.41|             1|
| 89|       22.7|     77|   4.69|0.890787333|  6.964|   5.589865| 12.9361|1256.083|             1|
| 76|     

90
26
//////////////////////////////////////////////////////////////////////////////////////////////////////////////////
(9,[0,1,2,5,7],[0.24460932768019383,0.09070866141732287,0.3967107279010938,0.1167901806391848,0.15118110236220478])
//////////////////////////////////////////////////////////////////////////////////////////////////////////////////
+--------------------+--------------+--------------+--------------------+----------+
|            features|Classification| rawPrediction|         probability|prediction|
+--------------------+--------------+--------------+--------------------+----------+
|[25.0,22.86,82.0,...|             1|[0.0,12.0,0.0]|       [0.0,1.0,0.0]|       1.0|
|[38.0,22.4996371,...|             2|[0.0,0.0,27.0]|       [0.0,0.0,1.0]|       2.0|
|[38.0,23.34,75.0,...|             1|[0.0,12.0,0.0]|       [0.0,1.0,0.0]|       1.0|
|[40.0,27.63605442...|             2|[0.0,0.0,27.0]|       [0.0,0.0,1.0]|       2.0|
|[43.0,31.25,103.0...|             2|[0.0,0.0,27.0]| 